<a href="https://colab.research.google.com/github/Bavalpreet/NLP/blob/master/nlp_vw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install nltk

In [0]:
import nltk
nltk.download('punkt')
nltk.download('nps_chat')
posts = nltk.corpus.nps_chat.xml_posts()[:20000]
print(posts)


def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features

featuresets = [(dialogue_act_features(post.text), post.get('class')) for post in posts]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package nps_chat to /root/nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!
[<Element 'Post' at 0x7f3122c08458>, <Element 'Post' at 0x7f3122c08d18>, ...]
0.6685606060606061


In [0]:
from nltk.corpus import nps_chat
import pandas as pd
data = []
for p in nps_chat.xml_posts():
    data.append({"class":p.get("class"), "text": p.text})
df = pd.DataFrame.from_dict(data)
df

,class,text
0,Statement,now im left with this gay name
1,Emotion,:P
2,System,PART
3,Greet,hey everyone
4,Statement,ah well
...,...,...
10562,Greet,hi 11-09-teensUser3
10563,System,JOIN
10564,Greet,"Hi, 11-09-teensUser197."
10565,nAnswer,"Not that I know of, 11-09-teensUser98"


In [0]:
# for vw i left runing here continue from 11-05-20
df["class"].unique()

array(['Statement', 'Emotion', 'System', 'Greet', 'Accept', 'Reject',
       'whQuestion', 'Continuer', 'ynQuestion', 'yAnswer', 'Bye',
       'Clarify', 'Emphasis', 'nAnswer', 'Other'], dtype=object)

In [0]:
!apt-get install libboost-program-options-dev zlib1g-dev libboost-python-dev vowpal-wabbit

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libboost-program-options-dev is already the newest version (1.65.1.0ubuntu1).
libboost-program-options-dev set to manually installed.
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
libboost-python-dev is already the newest version (1.65.1.0ubuntu1).
libboost-python-dev set to manually installed.
The following additional packages will be installed:
  libvw0
Suggested packages:
  vowpal-wabbit-doc
The following NEW packages will be installed:
  libvw0 vowpal-wabbit
0 upgraded, 2 newly installed, 0 to remove and 29 not upgraded.
Need to get 797 kB of archives.
After this operation, 3,034 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libvw0 amd64 8.5.0.dfsg1-1 [748 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 vowpal-wabbit amd64 8.5.0.dfsg1-1 [49.1 kB]
Fetched 79

In [0]:
!vw --help

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1


VW options:
  --random_seed arg                     seed random number generator
  --ring_size arg                       size of example ring

Update options:
  -l [ --learning_rate ] arg            Set learning rate
  --power_t arg                         t power value
  --decay_learning_rate arg             Set Decay factor for learning_rate 
                                        between passes
  --initial_t arg                       initial t value
  --feature_mask arg                    Use existing regressor to determine 
                                        which parameters may be updated.  If no
                                        initial_regressor given, also used for 
                                        initial weights.

Weight options:
  -i [ --initial_regressor ] arg        Initial regressor(s)
  --initial_weight arg                  Set all 

In [0]:
!pip install vowpalwabbit


In [0]:
from sklearn.preprocessing import LabelEncoder
import os
all_documents = df['text']
topic_encoder = LabelEncoder()
all_targets_mult = topic_encoder.fit_transform(df['class']) + 1

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_documents, test_documents, train_labels_mult, test_labels_mult = \
    train_test_split(all_documents, all_targets_mult, random_state=7)

def to_vw_format(document, label=None):
    return str(label or '') + ' |text ' + ' '.join(re.findall('\w{3,}', document.lower())) + '\n'

with open(os.path.join('/content/drive/My Drive/vw/train.vw'), 'w') as vw_train_data:
    for text, target in zip(train_documents, train_labels_mult):
        vw_train_data.write(to_vw_format(text, target))

with open(os.path.join('/content/drive/My Drive/vw/test.vw'), 'w') as vw_test_data:
    for text in test_documents:
        vw_test_data.write(to_vw_format(text))
        

In [0]:
# # Fitting a logistic regression for predicting the sentiment of a review
# !vw -d movie_reviews_train.vw --loss_function logistic -f movie_reviews_model.vw
%%time
!vw --oaa 15 $"/content/drive/My Drive/vw/train.vw" -f $"/content/drive/My Drive/vw/model_mult.vw" \
--loss_function=hinge

final_regressor = /content/drive/My Drive/vw/model_mult.vw
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = /content/drive/My Drive/vw/train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0       10        1        4
1.000000 1.000000            2            2.0       11       10       14
0.750000 0.500000            4            4.0       10       10        8
0.750000 0.750000            8            8.0        4       10        3
0.687500 0.625000           16           16.0       11       10        9
0.625000 0.562500           32           32.0       10       15        4
0.484375 0.343750           64           64.0       10       10        3
0.445312 0.406250          128          128.0       10       10        4
0.429688 0.414062          256          256.0       10  

In [0]:
%%time
!vw -i $"/content/drive/My Drive/vw/model_mult.vw" -t -d $"/content/drive/My Drive/vw/test.vw" \
-p $"/content/drive/My Drive/vw/test_predictions_mult.txt"

only testing
predictions = /content/drive/My Drive/vw/test_predictions_mult.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = /content/drive/My Drive/vw/test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
    n.a.     n.a.            1            1.0  unknown        5        2
    n.a.     n.a.            2            2.0  unknown        7        5
    n.a.     n.a.            4            4.0  unknown       11        2
    n.a.     n.a.            8            8.0  unknown       10        5
    n.a.     n.a.           16           16.0  unknown       10        4
    n.a.     n.a.           32           32.0  unknown       15        5
    n.a.     n.a.           64           64.0  unknown        7        2
    n.a.     n.a.          128          128.0  unknown       10        1
    n.a.     n.a.          256      

In [0]:
with open(os.path.join("/content/drive/My Drive/vw/test_predictions_mult.txt")) as pred_file:
    test_prediction_mult = [float(label) for label in pred_file.readlines()]

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(test_labels_mult, test_prediction_mult)

0.7369417108251325

In [0]:
from sklearn.metrics import confusion_matrix
M = confusion_matrix(test_labels_mult, test_prediction_mult)
for i in np.where(M[0,:] > 0)[0][1:]:
    print(df['class'][i], M[0,i])

Statement 2
Accept 1
System 26
System 3
System 6
Statement 1
